In [2]:
import plotly 
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
print(cf.__version__)
# Configure cufflings 
cf.set_config_file(offline=False, world_readable=True, theme='pearl')

0.12.1


In [3]:
import nltk
# nltk.download()
# !pip install autocorrect
from autocorrect import spell

In [4]:
import pandas as pd
import numpy as np
import gzip

In [5]:
# For monitoring duration of pandas processes
from tqdm import tqdm, tqdm_pandas

# To avoid RuntimeError: Set changed size during iteration
tqdm.monitor_interval = 0

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm_gui`, `tqdm_notebook`, optional kwargs, etc.)
tqdm.pandas(desc="Progress:")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
# can also groupby:
# df.groupby(0).progress_apply(lambda x: x**2)

### Data Loading

In [6]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

In [39]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [40]:
df = getDF('/Users/falehalrashidi/Downloads/reviews_Books_5.json.gz')

In [9]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5.0,Wonderful!,1355616000,"12 16, 2012"
1,A2S166WSCFIFP5,000100039X,"adead_poet@hotmail.com ""adead_poet@hotmail.com""","[0, 2]",This is one my must have books. It is a master...,5.0,close to god,1071100800,"12 11, 2003"
2,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""","[0, 0]",This book provides a reflection that you can a...,5.0,Must Read for Life Afficianados,1390003200,"01 18, 2014"
3,A1MOSTXNIO5MPJ,000100039X,Alan Krug,"[0, 0]",I first read THE PROPHET in college back in th...,5.0,Timeless for every good and bad time in your l...,1317081600,"09 27, 2011"
4,A2XQ5LZHTD4AFT,000100039X,Alaturka,"[7, 9]",A timeless classic. It is a very demanding an...,5.0,A Modern Rumi,1033948800,"10 7, 2002"


In [10]:
df1 = df[['reviewerID','asin','reviewText','helpful']]
df1.head()

,reviewerID,asin,reviewText,helpful
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]"
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]"
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]"
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]"
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]"


In [11]:
len(df1)

8898041

In [12]:
# Create new Column for the denominator
df2 = df1.assign(denom = df1['helpful'].progress_apply(lambda enum_denom:enum_denom[1]))
df2.head()

Progress:: 100%|██████████| 8898041/8898041 [00:06<00:00, 1388667.24it/s]


,reviewerID,asin,reviewText,helpful,denom
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]",0
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]",2
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]",0
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]",0
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]",9


In [13]:
# Create a Uniquekey Column
df3 = df2.assign(uniqueKey = df['reviewerID'].str.cat(df['asin'].values.astype(str), sep='##'))
df3.head()

,reviewerID,asin,reviewText,helpful,denom,uniqueKey
0,A10000012B7CGYKOMPQ4L,000100039X,Spiritually and mentally inspiring! A book tha...,"[0, 0]",0,A10000012B7CGYKOMPQ4L##000100039X
1,A2S166WSCFIFP5,000100039X,This is one my must have books. It is a master...,"[0, 2]",2,A2S166WSCFIFP5##000100039X
2,A1BM81XB4QHOA3,000100039X,This book provides a reflection that you can a...,"[0, 0]",0,A1BM81XB4QHOA3##000100039X
3,A1MOSTXNIO5MPJ,000100039X,I first read THE PROPHET in college back in th...,"[0, 0]",0,A1MOSTXNIO5MPJ##000100039X
4,A2XQ5LZHTD4AFT,000100039X,A timeless classic. It is a very demanding an...,"[7, 9]",9,A2XQ5LZHTD4AFT##000100039X


In [14]:
# Keep only the columns necessary for the normalisation
df4 = df3[['uniqueKey', 'reviewText']]
df4.head()

,uniqueKey,reviewText
0,A10000012B7CGYKOMPQ4L##000100039X,Spiritually and mentally inspiring! A book tha...
1,A2S166WSCFIFP5##000100039X,This is one my must have books. It is a master...
2,A1BM81XB4QHOA3##000100039X,This book provides a reflection that you can a...
3,A1MOSTXNIO5MPJ##000100039X,I first read THE PROPHET in college back in th...
4,A2XQ5LZHTD4AFT##000100039X,A timeless classic. It is a very demanding an...


The next step was necessary due to weird keyErrors that followed after trying to process the reviewText as a `pandas.DataFrame` and not as `pandas.Series`. After experimenting with both, I found that `pandas.Series.apply` is faster than `pandas.DataFrame.apply` and so I will hence work with `pandas.Series`. 

The assumption I require to make at this point before I follow is that `pandas` will not change the index of the reviews as those are being processed by my code and that in the end of my processing I will be able to re-associate those reviews with their **uniqueKey**. 

In [15]:
uniqueKey_series_df = df4[['uniqueKey']]
uniqueKey_series_df.head()

,uniqueKey
0,A10000012B7CGYKOMPQ4L##000100039X
1,A2S166WSCFIFP5##000100039X
2,A1BM81XB4QHOA3##000100039X
3,A1MOSTXNIO5MPJ##000100039X
4,A2XQ5LZHTD4AFT##000100039X


In [16]:
reviews_df = pd.DataFrame(df4['reviewText'].progress_apply(lambda review: review.split("\n")[0]))
reviews_df.head()

Progress:: 100%|██████████| 8898041/8898041 [00:08<00:00, 1010648.65it/s]


,reviewText
0,Spiritually and mentally inspiring! A book tha...
1,This is one my must have books. It is a master...
2,This book provides a reflection that you can a...
3,I first read THE PROPHET in college back in th...
4,A timeless classic. It is a very demanding an...


### Data Normalisation
<span style="color:red">**NOTICE:** As I am in a hurry to complete the first pipeline, I will only work with the first 100k reviews.</span>

* Tokenization <span style="color:blue"> DONE </span>
* Convert All Tokens to Lowercase <span style="color:blue"> DONE </span>
* Eliminate Punctuation <span style="color:blue"> DONE </span>
* Remove Stop Words <span style="color:blue"> DONE </span>
* Changing Numbers into Words <span style="color:blue"> DONE </span>
* Expand Abbreviations <span style="color:red"> NOT AS EASY AS I THOUGHT AND DOES NOT ADD MUCH VALUE (Ask Stasha's opinion)</span> 
* Correct Spelling <span style="color:red"> TOO SLOW (10h for 100k reviews)-->SO WONT DO</span>
* Substituting Tokens with Synonyms <span style="color:green"> TO DO</span>
* Semantical Marking of Negatives <span style="color:blue"> DONE (Ask Stasha's opinion) </span>

### Tokenization

In [17]:
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp_tokenize
tokenizer=RegexpTokenizer('[\'\w\-]+',gaps=False)

step_0_df = reviews_df['reviewText'][0:99999].progress_apply(lambda review: tokenizer.tokenize(review))
step_0_df.head()

Progress:: 100%|██████████| 99999/99999 [16:58<00:00, 98.14it/s]   


0    [Spiritually, and, mentally, inspiring, A, boo...
1    [This, is, one, my, must, have, books, It, is,...
2    [This, book, provides, a, reflection, that, yo...
3    [I, first, read, THE, PROPHET, in, college, ba...
4    [A, timeless, classic, It, is, a, very, demand...
Name: reviewText, dtype: object

### Convert Tokens to Lowercase

In [18]:
import re
import string

def convert_to_lowercase(review):

    for i in range(len(review)):
        review[i] = review[i].lower()
    return review

In [19]:
step_1_df = step_0_df.progress_apply(lambda review: convert_to_lowercase(review))
step_1_df.head()

Progress:: 100%|██████████| 99999/99999 [00:01<00:00, 58665.24it/s]


0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

### Eliminate Punctuation

In [20]:
import re
import string

def eliminate_punctuation(review, regex):
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    return new_review

In [21]:
regex=re.compile('[%s]' % re.escape(string.punctuation))

step_2_df = step_1_df.progress_apply(lambda review: eliminate_punctuation(review, regex))
step_2_df.head()

Progress:: 100%|██████████| 99999/99999 [00:05<00:00, 19761.26it/s]


0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

### Changing Numbers into Words

In [22]:
import inflect
p = inflect.engine()

def numStringToWord(review, p):

    for i in range(len(review)):
        if(review[i].isdigit()):
            review[i] = p.number_to_words(review[i])
    return review

In [23]:
step_3_df = step_2_df.progress_apply(lambda review: numStringToWord(review, p))
step_3_df.head()

Progress:: 100%|██████████| 99999/99999 [00:03<00:00, 27708.38it/s]


0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

### Correct Spelling
It turns out that this is a very expensive operation to run at this stage. In addition to this, the solution below by which I am able to substitue synonyms, also works as a spellchecker and hence applying a spell-checker directly and at this point is both non-efficient and seems to also be redundant. 

In [24]:
# from autocorrect import spell

# def spellCheck(review):

#     for i in range(len(review)):
#         review[i] = spell(review[i])
#     return review

In [25]:
# step_5_df = step_4_df.progress_apply(lambda review: spellCheck(review))
# step_5_df.head()

### Substituting Tokens with Synonyms
Replacing words with synonyms is a very tricky operation. In fact,synonyms are a huge and open area of work in natural language processing.

The problem is multifaceted and mostly derives from the fact that for any single word, and especially for adjectives, there is no one single adjective to replace it with. Suppose for instance that we rely on a solution based on the `PyDictionary` library. In the below example, you can clearly see that there are multiple words to be chosen for a certain noun: 

```python
>>> from PyDictionary import PyDictionary

>>> dictionary=PyDictionary()
print (dictionary.synonym("Life"))
['heart', 'growth', 'soul', 'activity', 'get-up-and-go']
```
It is therefore necessary that one defines a way by which a certain alternative is chosen from the resulting list. In `NLP` a list of synonyms is commonly reffered to as `synset`. 

A naive way to go about solving this problem is to choose the most common member in the set. The `nltk` will let you build a frequency table in just a few lines of code ([link](https://stackoverflow.com/questions/38233145/nltk-most-common-synonym-wordnet-for-each-word)). 
```python
>>> from nltk.corpus import brown
>>> freqs = nltk.FreqDist(w.lower() for w in brown.words())
>>> print(freqs["valued"])
14
```
But, in turn, one has to also figure out how to define the "most common member" in the set. This could be based on measuring the each members appearance frequence in the whole corpus. However, this would also be problematic for number of reasons; the main one relates to the semantical meaning of synonyms.

Synomyms are not semantically equal words in terms of their meaning. For instance, in the previous example, "growth" appears  to be a synonym for "life", yet, though related, the two words have different interpretations. Furthermore, if we take each word as a signleton and try to interpret its meaning irrespective of the context it is being used in, it is quite possible that our interpretation will deviade from the words intended meaning. The general point is that the context in which a word is used is also important. If we rely on the whole corpus to define the importance of a word over others then we may end up replacing words in book reviews, not based on how often those words are used with respect to the book at hand but with respect to the whole corpus of reviews for all kinds of books. 

A more sophisticated approach would focus on identifying word frequencies within the scope of reviews for a certain book. For that, one would need to:

1. Construct a set of review corpuses, one for every book;
2. Create a dictionary of frequencies of words for each of these corpuses, and;
3. Use the dictionary to choose the word with the highest frequency in a synset.

In addition to the above problem, one needs to tackle another difficulty. The point of replacing sysonyms is to reduce the vocabulary of a text. By compressing the vocabulary without losing meaning, you can save memory in cases such as frequency analysis and text indexing (https://en.wikipedia.org/wiki/Frequency_analysis). Vocabulary reduction can also increase the occurrence of significant collocations. But this can only be achieved as long a the same synonym is consistently chosen from a `synset` and the semantical meaning of the word it replaces is not distorted. Let us illustrate why this is a complex task. Following on the previous example:
```python
>>> print (dictionary.synonym("growth"))
['prosperity', 'success', 'advance', 'hike', 'rise']
>>> print (dictionary.synonym("success"))
['prosperity', 'advance', 'achievement', 'win', 'accomplishment']
>>> print (dictionary.synonym("prosperity"))
['wealth', 'success', 'accomplishment', 'riches', 'expansion']
>>> print (dictionary.synonym("advance"))
['forward', 'leading', 'prior', 'first', 'beforehand']
```

Notice, that "growth" and "success" don't share the exact same `synset`, so it is possible that while one synnonym is chosen for the first, another is chosen for the second. This incosistency will reduce the chances of minimising the corpus' vocabulary. Furhtermore, suppose that it just so happens that both words are replaced with one of the commonly shared synonyms--that is either 'prosperity' or 'advance'--then, if it is propserity, then propserity should not be replaced when encountered at all!

This problem, though complicated, can be handle with the developement of a well-thought algorithm. However, if the objective is to replace each and every word in the dataset, it will still fail as it is very likely that the solution will not be efficient or, worse, tractable. At the same time, it is quite possible that it will distort the text it originated from. To counter these problems, one can only apply this solution only on certain kinds of words, and specifically on adjectives. This approach offers two  significant advantages:

1. It reduces the application scope to something more tractable, and;
2. Minimises the risk of distorting meaning as adjectives and their `synsets` are more strictly defined and their meaning is unlikely to significantly deviate from the original word.

The above points can be justified simply by looking at the synsets of two random synonyms: 
```python
>>> print (dictionary.synonym("hard"))
['solid', 'strong', 'tough', 'concentrated', 'callous']
>>> print (dictionary.synonym("tough"))
['tenacious', 'vigorous', 'stiff', 'solid', 'hard']
```

As such, replacing of synonyms will be postponed until after POS-tagging is applied in the next section. 

### Replacing Negations with Antonyms
The opposite of synonym replacement is antonym replacement. An antonym is a word that has the opposite meaning of another word. This time, instead of creating custom word mappings, we can use WordNet to replace words with unambiguous antonyms. 

In [26]:
from nltk.corpus import wordnet

In [27]:
class AntonymReplacer(object):
    def replace(self, token, pos=None):
        antonyms = set()
        for syn in wordnet.synsets(token, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())
        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None
        
    def replace_negations(self, review):
        i, l = 0, len(review)
        tokens = []
        while i<l:
            token = review[i]
            if token == 'not' and i+1 <l:
                ant = self.replace(review[i+1])
                if ant:
                    tokens.append(ant)
                    i += 2
                    continue
            tokens.append(token)
            i += 1

        return tokens

In [28]:
# Lets see an example
replacer = AntonymReplacer()
replacer.replace("good")

In [29]:
replacer.replace("uglify")

'beautify'

In [30]:
review = ["lets","not","uglify","our","code"]

In [31]:
replacer.replace_negations(review)

['lets', 'beautify', 'our', 'code']

In [32]:
step_4_df = step_3_df.progress_apply(lambda review: replacer.replace_negations(review))
step_4_df.head()

Progress:: 100%|██████████| 99999/99999 [00:07<00:00, 12996.06it/s]


0    [spiritually, and, mentally, inspiring, a, boo...
1    [this, is, one, my, must, have, books, it, is,...
2    [this, book, provides, a, reflection, that, yo...
3    [i, first, read, the, prophet, in, college, ba...
4    [a, timeless, classic, it, is, a, very, demand...
Name: reviewText, dtype: object

### Remove Stopwords

In [33]:
### Remove Stop Words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(review):
    return [token for token in review if not token in stop_words]

In [34]:
step_5_df = step_4_df.progress_apply(lambda review: remove_stopwords(review))
step_5_df.head()

Progress:: 100%|██████████| 99999/99999 [00:01<00:00, 67163.46it/s]


0    [spiritually, mentally, inspiring, book, allow...
1    [one, must, books, masterpiece, spirituality, ...
2    [book, provides, reflection, apply, life, way,...
3    [first, read, prophet, college, back, 60s, boo...
4    [timeless, classic, demanding, assuming, title...
Name: reviewText, dtype: object

## JOIN normalised reviews with their original keys

In [35]:
tokenized_keyed_reviews = pd.concat([uniqueKey_series_df[0:99999], step_5_df], axis=1);
tokenized_keyed_reviews.head()

,uniqueKey,reviewText
0,A10000012B7CGYKOMPQ4L##000100039X,"[spiritually, mentally, inspiring, book, allow..."
1,A2S166WSCFIFP5##000100039X,"[one, must, books, masterpiece, spirituality, ..."
2,A1BM81XB4QHOA3##000100039X,"[book, provides, reflection, apply, life, way,..."
3,A1MOSTXNIO5MPJ##000100039X,"[first, read, prophet, college, back, 60s, boo..."
4,A2XQ5LZHTD4AFT##000100039X,"[timeless, classic, demanding, assuming, title..."


In [41]:
tokenized_keyed_reviews.to_csv("../data/interim/001_normalised_keyed_reviews_100k_sample.csv", sep='\t', header=True, index=False);

In [37]:
tokenized_keyed_reviews[0:99].to_csv("../data/interim/001_normalised_keyed_reviews_100_rows_sample.csv", sep='\t', header=True, index=False);

In [38]:
## END_OF_FILE